In [4]:
import numpy as np

In [5]:
## (days, Elfcount, Funds,  elf_hire_bit)

start = np.array([0, 12, 0, 0%2])

## (current state + elf_hire + allocation*weathertransformer

## Woods, Forest, Mountain

allocation = np.array([5, 10, 5])

## Woods*10 + good Forest*20 + Mountain*50 bad count - mountain

expected_transition = 2/3*np.array([[0,0,  15, 0],
                               [0,0,  20, 0],
                               [0,-0.5,  50, 0]]
                               
                              )


print(expected_transition)

good_transition = np.array([[0,0,  10, 0],
                               [0,0,  20, 0],
                               [0,0,  50, 0]])

bad_transition = np.array([[0,0,  10, 0],
                               [0,0,  0, 0],
                               [0,-1,  0, 0]])


[[  0.           0.          10.           0.        ]
 [  0.           0.          13.33333333   0.        ]
 [  0.          -0.33333333  33.33333333   0.        ]]


We can still create a bot and see what it does

In [6]:
def random_bot(state, transition = good_transition):
    
    weather = np.random.randint(0,6)
    if weather >3:
        transition = bad_transition
    ##hiring
    if state[3]==0:
        hire_total = np.random.randint(0,np.floor(state[2]/75)+1)
        print(hire_total)
        state += np.array([0,hire_total,0,1])
    ##allocating
    elif state[3]==1:
        elf_count = state[1]
        woods = np.random.randint(0,elf_count+1)
        forest = np.random.randint(0,elf_count-woods+1)
        mountain = np.random.randint(0,elf_count-woods-forest+1)
    
        allocation = np.array([woods,forest,mountain])
    
        state = state + np.dot(allocation, transition) + np.array([1,0,0,-1])
    return state
    
    

In [7]:
start = np.array([0, 12, 0, 0])
state = start
while state[0]<10:    
    print(state)
    state = random_bot(state)


[ 0 12  0  0]
0
[ 0 12  0  1]
[ 1  9 20  0]
0
[ 1  9 20  1]
[  2   9 100   0]
1
[  2  10 100   1]
[  3  10 150   0]
1
[  3  11 150   1]
[  4  11 330   0]
4
[  4  15 330   1]
[  5  15 540   0]
3
[  5  18 540   1]
[  6  18 720   0]
1
[  6  19 720   1]
[   7   19 1320    0]
11
[   7   30 1320    1]
[   8   30 1630    0]
2
[   8   32 1630    1]
[   9   23 1830    0]
1
[   9   24 1830    1]


Solving the Elf Game

In [8]:
## hiring function generates possible hiring options for a state
def hiring(state):
    total = 0
    #print(state[2]//75+1)
    while total<= state[2]//75:
                
        hire_total = total
        #print(state)
        test_state = state + np.array([0,hire_total,-hire_total*75,1])
        #print(test_state)
        yield hire_total, test_state
        total+=1
        
def hiring_tight(state):
    total = 0
    #print(state[2]//75+1)
    while total<= state[2]//75:
                
        hire_total = total
        #print(state)
        test_state = state + np.array([0,hire_total,-hire_total*75,1])
        #print(test_state)
        yield test_state
        total+=1

        
def hire_number(state):
    total = 0
    while total <= state[2]//75:
        yield total
        total+=1

def hired_state(state, hire_total):
    hired = state + np.array([0,hire_total,-hire_total*75,1])
    return hired

## allocator function generates all the possible allocations of elves to Woods, Forest, Mountain
def allocator(state):        
    elf_count = state[1]
    
    elves = 0
    while elves <= elf_count:    
        woods = elves
        remaining_elves = elf_count-woods
        forest = 0

        while forest <=remaining_elves:

            
            mountain = remaining_elves-forest
            allocation = np.array([woods,forest,mountain])
            yield allocation
            forest+=1
        elves+=1
        

In [9]:
floor_vec = np.vectorize(lambda x: int(np.floor(x)))

##result_generator links the evaluated state and the allocation, so that we can track back on it

def result_generator(state):
    
    for allocation in allocator(state):
    
        evaluated_state = state + np.dot(allocation, expected_transition) + np.array([1,0,0,-1])
        evaluated_state = floor_vec(evaluated_state)
        yield (evaluated_state, allocation)

def result(state, allocation):
    evaluated_state = state + np.dot(allocation, expected_transition) + np.array([1,0,0,-1])
    evaluated_state = floor_vec(evaluated_state)
    return evaluated_state


OPT(i,j,k,b) itself can be defined as a
function which recursively calls itself
for new states (i',j',k',b') as needed,
but such that, *before* doing a recursive
call on (i',j',k',b'),   we CHECK in
a lookup table whether the value/action
solution for (i',j',k',b') has already
been computed before (i.e., "memoized"),
in which case we do not need to
make the recursive call, and can
simply use that answer.

Moreover, once we do compute an answer
during the computation for some state
(i',j',k',d'),  we insert that answer
into the lookup table for future use.

(The lookup table can be an efficient
 hash table, or array, or other efficient
 data structure suitable for this purpose.)

Now, of course the key question is
"what is the nature of the recursive procedure
that defines OPT(i,j,k,b)".

This simply follows the definition of the
(partially probabilistic) "game tree" that I described in my prior
email.

In [21]:
start = np.array([0, 3, 75, 0])
memo_table={'End':np.array([0,0,0,0]),'penultimate':start,'pen_action':np.array([0,0,0])}

def recursive_tree(state,gamelength):
    
    state_key = np.array2string(state)
     
    if state_key in memo_table.keys():
        return memo_table
    elif state[3] == 0 and state[0]<gamelength:
        
        for h in hire_number(state):
            hired = hired_state(state,h)
            hired_key = np.array2string(hired)
            if hired_key in memo_table.keys():
                print('winning')
                
                memo_table[state_key]=h
            else:
                recursive_tree(hired,gamelength)
    elif state[3] == 1 and state[0]<gamelength-1:
        
        for r in result_generator(state):
            result_key = np.array2string(r[0])
            if result_key in memo_table.keys():
                print('interesting')
                memo_table[state_key]=r[1]
                recursive_tree
            else:
                recursive_tree(r[0],gamelength)
                

           
    else:
        
        max_state=memo_table['End']
        penstate = memo_table['penultimate']
        best_action = memo_table['pen_action']
        for r in result_generator(state):
            r_key = np.array2string(r[0])
            
            
            if r[0][2]>max_state[2]:
                #print('currently max state because r[0][2]%s>max_state[2]%s' %(r[0][2],max_state[2]))
                max_state=r[0]
                best_action = r[1]
                penstate = state
                
                
                #print(new_max_state)    
            memo_table['End']=max_state
            memo_table['penultimate']= penstate
            memo_table['pen_action']=best_action
        
    #memo_table[np.array2string(memo_table['penultimate'])]=memo_table['pen_action']
    #recursive_tree(memo_table['penultimate'],gamelength)    
        







In [22]:
recursive_tree(start,4)
memo_table[np.array2string(memo_table['penultimate'])]=memo_table['pen_action']


In [23]:
memo_table

{'End': array([  4,   1, 332,   0]),
 '[  3   2 266   1]': array([0, 0, 2]),
 'pen_action': array([0, 0, 2]),
 'penultimate': array([  3,   2, 266,   1])}

So recursive tree has identified the penultimate state and the end state with the optimal expected payoff.

Now we need to trace back the previous states and actions so that we know how to get there.

In [26]:
while np.array2string(start) not in memo_table.keys():
    recursive_tree(start,4)
    
memo_table

winning
interesting
winning
interesting
winning
interesting
winning


{'End': array([  4,   1, 332,   0]),
 '[  2   3 166   0]': 0,
 '[  2   3 166   1]': array([0, 0, 3]),
 '[  3   2 266   0]': 0,
 '[  3   2 266   1]': array([0, 0, 2]),
 '[ 0  3 75  0]': 1,
 '[ 1  4 53  0]': 0,
 '[ 1  4 53  1]': array([0, 1, 3]),
 '[0 4 0 1]': array([0, 4, 0]),
 'pen_action': array([0, 0, 2]),
 'penultimate': array([  3,   2, 266,   1])}

In [28]:
## trying to organise this starting with the first day then the first action...

keys = [k for k in memo_table.keys()]
for k in sorted(keys):
    print(k)
    print(memo_table[k])

End
[  4   1 332   0]
[  2   3 166   0]
0
[  2   3 166   1]
[0 0 3]
[  3   2 266   0]
0
[  3   2 266   1]
[0 0 2]
[ 0  3 75  0]
1
[ 1  4 53  0]
0
[ 1  4 53  1]
[0 1 3]
[0 4 0 1]
[0 4 0]
pen_action
[0 0 2]
penultimate
[  3   2 266   1]
